## Transforming Legacy Data to upload into LibInsight

In [213]:
# Import libraries and functions

import polars as pl
from polars import col, lit, Int64, when
from great_tables import GT

In [ ]:
# Read in csv from file path

path = ''
df_old = pl.read_csv(path)

### Data Transformations

In [215]:
# Rename columns

df_old = df_old.rename({
    'When did this question happen?':'Start Date',
    'How was the question asked?':'Where did the interaction take place?',
    'Was this question by walk-in or appointment?':'Question Source',
    'What kind of question was it?':'Question Type',
    'Time spent on interaction':'Duration',
    'Any other notes:':'Any other notes'
})




# Add in new columns with null values

df_old = df_old.with_columns([
                            (lit(None).alias('If referred, referred to:')), 
                            (lit(None).alias('Notes: sources consulted / recommended, problems, etc.')), 
                            (lit(None).alias('Question')), 
                            (lit(None).alias('Who Asked?')),  
                            (lit(None).alias('Internal Notes')), 
                            (lit(None).alias('Entered By'))
                    
                              ])




# Alter Data in Columns

col_name1 = "Where did the interaction take place?"

df_old = df_old.with_columns([
    when(col(col_name1) == "In Office (Morning)").then(lit("My Office"))
    .when(col(col_name1) == "In Office (Afternoon)").then(lit("My Office"))
    .when(col(col_name1) == "Roving").then(lit("Roaming"))
    .when(col(col_name1) == "By Telephone").then(lit("By Telephone/videoconference"))
    .otherwise(col(col_name1))
    .alias(col_name1)
])




col_name2 = 'Question Type'

df_old = df_old.with_columns([
    when(col(col_name2) == "Turnitin").then(lit("Other"))
    .when(col(col_name2) == "Trademark").then(lit("Other"))
    .when(col(col_name2) == "Referred to IT Support Desk").then(lit("Computer Assistance (printing, paper, broken links, Microsoft suite, scanner, etc.)"))
    .when(col(col_name2) == "Copyright").then(lit("Other"))
    .when(col(col_name2) == "Paper jam").then(lit("Computer Assistance (printing, paper, broken links, Microsoft suite, scanner, etc.)"))
    .when(col(col_name2) == '''Reference: All other ILL (besides "where do I pick up"), database questions, looking up a book in catalog, citation, can't find book on shelf, writing help''').then(lit("Research Help"))
    .when(col(col_name2) == "Directional: Bathroom, stapler, if they already have a call #, where to pick up ILL book/reserve item").then(lit("General Library Info (directions, hours, policies, etc.)"))
    .when(col(col_name2) == "Computer: Printer/-ing, paper, broken links, Word/Powerpoint settings, scanner").then(lit("Computer Assistance (printing, paper, broken links, Microsoft suite, scanner, etc.)"))
    .when(col(col_name2) == "Question about an assignment from an instructional session.").then(lit("Research Help"))
    .otherwise(col(col_name2))
    .alias(col_name2)
])





col_name3 = "Librarian"

df_old = df_old.with_columns([
    when(col(col_name3) == "Breton").then(lit("A Breton"))
    .when(col(col_name3) == "Lieggi").then(lit("T Lieggi"))
    .when(col(col_name3) == "Hill").then(lit("D Hill"))
    .when(col(col_name3) == "Schoenfelder").then(lit("Other"))
    .otherwise(col(col_name3))
    .alias(col_name3)
])




col_name4 = "Duration"

df_old = df_old.with_columns([
    when(col(col_name4) == "0-3 minutes").then(lit(2))
    .when(col(col_name4) == "4-10 minutes").then(lit(8))
    .when(col(col_name4) == "11-30 minutes").then(lit(20))
    .when(col(col_name4) == "30-60 minutes").then(lit(45))
    .when(col(col_name4) == "60+ minutes").then(lit(60))
    .otherwise(col(col_name4))
    .alias(col_name4)
])

df_old = df_old.with_columns([col("Duration").cast(Int64).alias("Duration")]) # Set column to integer


df_old = df_old.with_columns([
    # Step 1: Parse string to date
    pl.col("Start Date").str.strptime(pl.Date, "%m/%d/%Y").alias("Start Date")
])


# Order columns into the template needed by LibInsight

column_order = [
    "Start Date",
    "Internal Notes",
    "Entered By",
    "Any other notes",
    "Class # and Section (ex: HIST 226 A)",
    "Class Title",
    "Duration",
    "If referred, referred to:",
    "Instructor (First Initial. Last Name - ex: S. Holstein)",
    "Librarian",
    "Notes: sources consulted / recommended, problems, etc.",
    "Question",
    "Question Source",
    "Question Type",
    "Where did the interaction take place?",
    "Who Asked?"
]

df_new = df_old.select(column_order)

### Export as CSV

In [216]:
df_new.write_csv('LegacyData.csv')

### Group by Librarian and create a table with great_tables

In [217]:
# Group once, aggregate both count and sum
grouped = (
    df_new.group_by("Librarian")
    .agg([
        pl.col("Librarian").count().alias("Consultations"),
        pl.col("Duration").sum().alias("Minutes")
    ])
    .sort("Consultations")
)



# Build GT table
table =(
    GT(grouped)
    .tab_header(
        title="Librarians, are they working?",
        subtitle="A look at total consultations and reference minutes"
    )
    .tab_stub(rowname_col="Librarian")
    .tab_source_note(source_note="Source: Self Reported Reference Data for SY 2024.")
    .tab_source_note(
        source_note=("Analyst: Redacted")
    )
    .tab_stubhead(label="Librarian")
    .fmt_integer(columns="Minutes")
)

# Show table
#table.show()